In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip

--2023-09-24 11:59:06--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.39.82, 13.249.39.6, 13.249.39.89, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.39.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  81.6MB/s    in 89s     

2023-09-24 12:00:35 (111 MB/s) - ‘wiki.en.zip’ saved [10356881291/10356881291]



In [ ]:
!unzip wiki.en.zip

Archive:  wiki.en.zip
  inflating: wiki.en.vec             
  inflating: wiki.en.bin             


In [ ]:
!wget http://alfonseca.org/pubs/ws353simrel.tar.gz

--2023-09-24 12:05:36--  http://alfonseca.org/pubs/ws353simrel.tar.gz
Resolving alfonseca.org (alfonseca.org)... 162.215.249.67
Connecting to alfonseca.org (alfonseca.org)|162.215.249.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5460 (5.3K) [application/x-gzip]
Saving to: ‘ws353simrel.tar.gz.1’

ws353simrel.tar.gz. 100%[===================>]   5.33K  --.-KB/s    in 0s      

2023-09-24 12:05:36 (439 MB/s) - ‘ws353simrel.tar.gz.1’ saved [5460/5460]



In [ ]:
!tar -zxvf ws353simrel.tar.gz

wordsim353_sim_rel/wordsim353_agreed.txt
wordsim353_sim_rel/wordsim353_annotator1.txt
wordsim353_sim_rel/wordsim353_annotator2.txt
wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt
wordsim353_sim_rel/wordsim_similarity_goldstandard.txt


In [ ]:
!wget https://fh295.github.io/simlex.html

--2023-09-24 12:12:49--  https://fh295.github.io/simlex.html
Resolving fh295.github.io (fh295.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fh295.github.io (fh295.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7073 (6.9K) [text/html]
Saving to: ‘simlex.html’

simlex.html         100%[===================>]   6.91K  --.-KB/s    in 0s      

2023-09-24 12:12:49 (66.4 MB/s) - ‘simlex.html’ saved [7073/7073]



In [ ]:
import pandas as pd

similarity_data = pd.read_csv("/content/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt", sep='\t', header=None, names=["Word1", "Word2", "Human_Similarity"])

relatedness_data = pd.read_csv("/content/wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt", sep='\t', header=None, names=["Word1", "Word2", "Human_Relatedness"])


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=4ee3e4b69c54dede16e30cf5e96d3aacefe6ef6885ae3bae8168139e85260ed5
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext.util
ft = fasttext.load_model('wiki.en.bin')

In [ ]:
def get_word_vectors(word_list):
    word_vectors = []
    for word in word_list:
        vector = ft.get_word_vector(word)
        word_vectors.append(vector)
    return word_vectors

similarity_data["Vector1"] = get_word_vectors(similarity_data["Word1"])
similarity_data["Vector2"] = get_word_vectors(similarity_data["Word2"])

relatedness_data["Vector1"] = get_word_vectors(relatedness_data["Word1"])
relatedness_data["Vector2"] = get_word_vectors(relatedness_data["Word2"])


In [ ]:
from scipy.spatial.distance import cosine

def compute_similarity(row):
    vector1 = row["Vector1"]
    vector2 = row["Vector2"]

    if vector1 is None or vector2 is None:
        return None

    return 1 - cosine(vector1, vector2)


similarity_data["Model_Similarity"] = similarity_data.apply(compute_similarity, axis=1)

relatedness_data["Model_Relatedness"] = relatedness_data.apply(compute_similarity, axis=1)


In [ ]:
# print(similarity_data.head())

In [ ]:
from scipy.stats import spearmanr

similarity_corr, _ = spearmanr(similarity_data["Human_Similarity"], similarity_data["Model_Similarity"])
print("Спирмен для similarity: " + str(similarity_corr))

relatedness_corr, _ = spearmanr(relatedness_data["Human_Relatedness"], relatedness_data["Model_Relatedness"])
print("Спирмена для relatedness: " + str(relatedness_corr))


Спирмен для similarity: 0.7709829765515606
Спирмена для relatedness: 0.6000385071915036


In [ ]:
import pandas as pd

simlex_data = pd.read_csv("SimLex-999.txt", sep='\t')
simlex_data = simlex_data[["word1", "word2", "SimLex999"]]

print(simlex_data.head())


   word1        word2  SimLex999
0    old          new       1.58
1  smart  intelligent       9.20
2   hard    difficult       8.77
3  happy     cheerful       9.55
4   hard         easy       0.95


In [ ]:
simlex_data["Vector1"] = get_word_vectors(simlex_data["word1"])
simlex_data["Vector2"] = get_word_vectors(simlex_data["word2"])


In [ ]:
simlex_data["Model_Simlex"] = simlex_data.apply(compute_similarity, axis=1)


In [ ]:
simlex_corr, _ = spearmanr(simlex_data["SimLex999"], simlex_data["Model_Simlex"])
print("Спирмен для Simlex: " + str(simlex_corr))


Спирмен для Simlex: 0.38033840459701396


In [ ]:
word_pairs = [("old", "new"), ("hard", "easy"), ("short", "long")]

results = []

for word1, word2 in word_pairs:
    selected_row = simlex_data[(simlex_data["word1"] == word1) & (simlex_data["word2"] == word2)]
    if not selected_row.empty:
        similarity = selected_row.apply(compute_similarity, axis=1).values[0]
        results.append({"Word1": word1, "Word2": word2, "Model_Similarity": similarity})

for result in results:
    print(result)


{'Word1': 'old', 'Word2': 'new', 'Model_Similarity': 0.3278508186340332}
{'Word1': 'hard', 'Word2': 'easy', 'Model_Similarity': 0.5655667781829834}
{'Word1': 'short', 'Word2': 'long', 'Model_Similarity': 0.6534804105758667}
